Setup


In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("DataTask_Scores.xlsx")
df

,SubjID,Session,Score
0,Ex1,1,20
1,Ex1,5,22
2,Ex1,6,19
3,Ex1,11,14
4,Ex1,15,10
5,Ex2,2,30
6,Ex2,5,24
7,Ex2,11,20
8,Ex3,3,30
9,Ex3,6,33


In [3]:
df.describe()

,Session,Score
count,13.000000,13.000000
mean,6.846154,23.153846
std,4.180234,6.693855
min,1.000000,10.000000
25%,4.000000,20.000000
50%,6.000000,24.000000
75%,10.000000,30.000000
max,15.000000,33.000000


In [4]:
#This will need to be modified if we care about more potential sessions             
list_values_alloted = [1,5,10,15]
students = df.SubjID.unique()
threshold_for_caring = 1

Process for Relevant Info

In [5]:
#These threshold calculations would need to be more robust by using less <= and >= operators if we want to expand this in the future
earlier = [i - threshold_for_caring for i in list_values_alloted]
later = [i + threshold_for_caring for i in list_values_alloted]
almost_alloted = earlier + later
df_ideal = df[df['Session'].isin(list_values_alloted)]
df_ideal


,SubjID,Session,Score
0,Ex1,1,20
1,Ex1,5,22
4,Ex1,15,10
6,Ex2,5,24
11,Ex3,10,25
12,Ex3,10,24


In [6]:
#Aggegate in case of multiple subj/session combination inputs
df_ideal = df_ideal.groupby(['SubjID', 'Session']).agg({'Score': 'mean'}).reset_index()
df_ideal

,SubjID,Session,Score
0,Ex1,1,20.0
1,Ex1,5,22.0
2,Ex1,15,10.0
3,Ex2,5,24.0
4,Ex3,10,24.5


In [7]:
df_earlier = df[df['Session'].isin(earlier)]
df_earlier['Session'] = df_earlier['Session'] + threshold_for_caring
df_earlier


C:\Users\hunte\AppData\Local\Temp\ipykernel_21768\2201229561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_earlier['Session'] = df_earlier['Session'] + threshold_for_caring


,SubjID,Session,Score
10,Ex3,5,30


In [8]:
df_later = df[df['Session'].isin(later)]
df_later['Session'] = df_later['Session'] - threshold_for_caring
df_later

C:\Users\hunte\AppData\Local\Temp\ipykernel_21768\839358168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_later['Session'] = df_later['Session'] - threshold_for_caring


,SubjID,Session,Score
2,Ex1,5,19
3,Ex1,10,14
5,Ex2,1,30
7,Ex2,10,20
9,Ex3,5,33


In [9]:
combined_almost = pd.concat([df_earlier,df_later])
combined_almost

,SubjID,Session,Score
10,Ex3,5,30
2,Ex1,5,19
3,Ex1,10,14
5,Ex2,1,30
7,Ex2,10,20
9,Ex3,5,33


In [10]:
df_Averaged = combined_almost.groupby(['SubjID', 'Session']).agg({'Score': 'mean'}).reset_index()
df_Averaged

,SubjID,Session,Score
0,Ex1,5,19.0
1,Ex1,10,14.0
2,Ex2,1,30.0
3,Ex2,10,20.0
4,Ex3,5,31.5


In [11]:
#We need to remove the values that are in the 'additional' category if we already have the ideal version provided
merged_df = df_ideal.merge(df_Averaged, on = ['SubjID','Session'], how = 'right', indicator = True)

anti_joined_df = merged_df[merged_df['_merge'] == 'right_only']

anti_joined_df['Score'] = anti_joined_df['Score_y']
relevant_additonal = anti_joined_df[['SubjID','Session','Score']]
relevant_additonal

C:\Users\hunte\AppData\Local\Temp\ipykernel_21768\533100099.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anti_joined_df['Score'] = anti_joined_df['Score_y']


,SubjID,Session,Score
1,Ex1,10,14.0
2,Ex2,1,30.0
3,Ex2,10,20.0
4,Ex3,5,31.5


In [12]:
#This is what it all looks like cleaned up before changing how we want the data to look
relevant_modified_df = pd.concat([df_ideal,relevant_additonal])
relevant_modified_df = relevant_modified_df.sort_values(by=['SubjID'])
relevant_modified_df


,SubjID,Session,Score
0,Ex1,1,20.0
1,Ex1,5,22.0
2,Ex1,15,10.0
1,Ex1,10,14.0
3,Ex2,5,24.0
2,Ex2,1,30.0
3,Ex2,10,20.0
4,Ex3,10,24.5
4,Ex3,5,31.5


Transform the data into Desired Output

In [13]:
#Time to hash the info so we can identify it easily
dist_scores = {}
for index, row in relevant_modified_df.iterrows():
    hash = f"{row['SubjID']} {row['Session']}"
    dist_scores[hash] = row['Score']
dist_scores

{'Ex1 1': 20.0,
 'Ex1 5': 22.0,
 'Ex1 15': 10.0,
 'Ex1 10': 14.0,
 'Ex2 5': 24.0,
 'Ex2 1': 30.0,
 'Ex2 10': 20.0,
 'Ex3 10': 24.5,
 'Ex3 5': 31.5}

In [14]:
#Finally lets just change it based on the sessions that we want
final_created_df = pd.DataFrame()
final_created_df['Students'] = students

for count,session in enumerate(list_values_alloted):
    list_for_session = ['not_found'] * len(students)
    for count2,student in enumerate(students):
        try:
            list_for_session[count2] = dist_scores[f'{student} {session}']
        except:
            pass
    final_created_df[f'Score at Session {session}'] = list_for_session
final_created_df


,Students,Score at Session 1,Score at Session 5,Score at Session 10,Score at Session 15
0,Ex1,20.0,22.0,14.0,10.0
1,Ex2,30.0,24.0,20.0,not_found
2,Ex3,not_found,31.5,24.5,not_found


There could be potential issues with the data. I took care of some of the issues currently present by aggregating the duplicate relevant sessions using averages. 

Another issue is if we have invalid data types entered in the original data. We would first need to have more preprocessing to eliminate the invalid results before calculating or we will get errors. 

Further, there is some dropped information for sessions that weren't within the threshold for relevance. If that data is relevant in anyway, it needs to be included in some other method!

This was really easy, definitely confident in my pandas and python data processing haha.